In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import keras
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

from xception import Xception, preprocess_input

Using TensorFlow backend.


In [3]:
data_dir = '/home/ubuntu/data/'

In [4]:
data_generator = ImageDataGenerator(
    rotation_range=30, 
    zoom_range=0.3,
    horizontal_flip=True, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.001,
    channel_shift_range=0.1,
    fill_mode='reflect',
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

data_generator_val = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train_no_resizing', 
    target_size=(299, 299),
    batch_size=64
)

val_generator = data_generator_val.flow_from_directory(
    data_dir + 'val', shuffle=False,
    target_size=(299, 299),
    batch_size=64
)

Found 16980 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


# Model

In [5]:
model = Xception(weight_decay=1e-5)

# Training

In [6]:
model.compile(
    optimizer=optimizers.SGD(lr=1e-2, momentum=0.9, nesterov=True), 
    loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy']
)

In [7]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=266, epochs=30, verbose=1,
    callbacks=[
        ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, epsilon=0.007),
        EarlyStopping(monitor='val_acc', patience=4, min_delta=0.01)
    ],
    validation_data=val_generator, validation_steps=80, workers=4
)

Epoch 1/30
266/266 [==============================] - 163s - loss: 3.9762 - acc: 0.3249 - top_k_categorical_accuracy: 0.4612 - val_loss: 3.2910 - val_acc: 0.4666 - val_top_k_categorical_accuracy: 0.7197
Epoch 2/30
266/266 [==============================] - 161s - loss: 2.2335 - acc: 0.6227 - top_k_categorical_accuracy: 0.8145 - val_loss: 1.8696 - val_acc: 0.6570 - val_top_k_categorical_accuracy: 0.8789
Epoch 3/30
266/266 [==============================] - 161s - loss: 1.5226 - acc: 0.7112 - top_k_categorical_accuracy: 0.8911 - val_loss: 1.3368 - val_acc: 0.7346 - val_top_k_categorical_accuracy: 0.9189
Epoch 4/30
266/266 [==============================] - 161s - loss: 1.2231 - acc: 0.7527 - top_k_categorical_accuracy: 0.9124 - val_loss: 1.0939 - val_acc: 0.7707 - val_top_k_categorical_accuracy: 0.9289
Epoch 5/30
266/266 [==============================] - 161s - loss: 1.0554 - acc: 0.7792 - top_k_categorical_accuracy: 0.9242 - val_loss: 0.9827 - val_acc: 0.7822 - val_top_k_categorical_ac

# Results

In [9]:
model.evaluate_generator(val_generator, 80)

[0.70807909807190295, 0.83027343750000004, 0.953125]

In [10]:
model.save_weights('xception_weights.hdf5')